## FINANCIAL DATA

MODULE 3 | LESSON 1


---

# CALCULATING PORTFOLIO RISK/RETURN STATISTICS

|  |  |
|:---|:---|
|**Reading Time** |  40 minutes |
|**Prior Knowledge** | Simple Stock Returns, Variance, Python, Linear Algebra, Matrix Multiplication  |
|**Keywords** | Portfolio Return, Variance, Sharpe Ratio |

---

*The previous module had a focus on the individual security. For this lesson, we take what we have learned previously and apply it to a portfolio setting using a basket of stocks/crypto.*

In [1]:
import datetime
import math

import numpy as np
import pandas_datareader.data as web
from IPython.display import VimeoVideo

## 1. Portfolio Returns
Up to this point in the course, we've spent our time analyzing individual securities. Today, we perform a similar analysis except using a portfolio of assets. We will first quickly recap how to **determine return on investment for a single asset, General Electric (GE)**.

### 1.1 Single Asset Return Recap
We will assume we bought 100 shares 10 years ago. To determine the cash return, we just need to use the following formula:

$r_f = (p_f - p_i) * 100$

In [2]:
start = datetime.date(2011,11,29)
end = datetime.date(2021,11,28)
# start = datetime.date.today() - datetime.date.timedelta(365*10)
# end = datetime.date.today()

In [3]:
prices = web.DataReader(["GE"], "yahoo" , start, end, )["Adj Close"]

In [4]:
start = datetime.date.today() - datetime.timedelta(365*10)
end = datetime.date.today()
prices = web.DataReader(["GE"], "yahoo", start, end)["Adj Close"]
initialPrice = prices.GE[0]
finalPrice = prices.GE[-1]
cashReturn = (finalPrice - initialPrice)*100
print(
    f"With an initial investment of the ${np.round(initialPrice*100,2)}, the cash return of this investment would be ${np.round(finalPrice*100,2)} - ${np.round(initialPrice*100,2)} = ${np.round(cashReturn,2)}")

With an initial investment of the $12605.81, the cash return of this investment would be $6307.0 - $12605.81 = $-6298.81


### 1.2 How to Calculate Return with a Basket of Assets
We’ve now gone over returns of a single asset many times during this course. It’s time to apply similar logic to a basket of multiple assets. This will once again be best illustrated with an example. The following calculations can easily be extended to n number of securities, but to keep it simple, we will use two for our example. For this, we need to make some basic assumptions:

* 2 stocks
    * Meta—We will refer to it as its previous name from here on out: Facebook (FB) 
    * Chipotle (CMG)
* Bought 100 shares of each five years ago
* Goal: Calculate percentage return obtained by the portfolio at the end of the period

To start, we need to calculate the weights of each asset at the start of the period:

In [5]:
# Define all initial variables
# start = datetime.date.today()-datetime.timedelta(365*5)
# end = datetime.date.today()
#start = datetime.date(2016, 11, 29)
#end = datetime.date(2021, 11, 28)
prices = web.DataReader(["FB", "CMG"], "yahoo", start, end)["Adj Close"]
initialFB = prices.FB[0]
initialCMG = prices.CMG[0]
finalFB = prices.FB[-1]
finalCMG = prices.CMG[-1]
FBweight = initialFB / (initialFB+initialCMG)
CMGWeight =  initialCMG / (initialFB + initialCMG)

print(
    f"We have an initial investment in FB of ${np.round(initialFB * 100,2)} and in CMG ${np.round(initialCMG *100)}")

We have an initial investment in FB of $nan and in CMG $38346.0


In [6]:
print(
    f"This would make the weights {FBweight} and {CMGWeight} for FB and CMG respectively.")

This would make the weights nan and nan for FB and CMG respectively.


### 1.3 Final Returns

In order to calculate the final portfolio percentage returns, we need to find the returns of each asset individually, multiply the return by the weight in our portfolio, and then add them together. The formula to calculate this for two assets is:

$Portfolio_{Return} = w_1R_1 + w_2R_2$

In [7]:
returnFB = 100 * (finalFB - initialFB) / initialFB
returnCMG = 100 * (finalCMG - initialCMG) / initialCMG

In [8]:
print(f"This return over this periof for Facebook is {np.round(returnFB,2)} (FB) and {np.round(returnCMG,2)} (CMG)")

This return over this periof for Facebook is nan (FB) and 241.0 (CMG)


Adding these weighted returns together gives us a portfolio return of 41.14+250.35 = 291.49% 

Our portfolio would have returned 291.49% over the last five years, assuming we invested in both assets on the same starting date and with our weights.

In the first video of this lesson, we show how to transition from calculating an individual security's returns to calculating the returns of a portfolio of assets.

In [9]:
VimeoVideo("706655699", h="1a478ba2bf", width=600)

## 2. Calculating Portfolio Variance
This was discussed at a high level during the Financial Markets course, but here, we will show how to **calculate variance in Python with empirical data**. While returns are important, investors are also concerned about risk or volatility. We will use variance of returns to determine risk. Once again, we will use an example, using the same data as before to drive the point home. A bit of linear algebra is needed, but luckily, we have Python to do the tough calculations for us. 

If we were to calculate this for two assets using pencil and paper, we could use the following formula:

$\textrm{Portfolio variance} = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2Cov_{1,2}$

Where:

* $w_1$ = the portfolio weight of the first asset

* $w_2$ = the portfolio weight of the second asset

* $\sigma_1$ = the standard deviation of the first asset

* $\sigma_2$ = the standard deviation of the second asset

* $Cov_{1,2} = \textrm{the covariance of the two assets, which can thus be expressed as } \rho_{(1,2)}\sigma_1\sigma_2, \textrm{where }\rho_{(1,2)} \textrm{is the correlation coefficient between the two assets}$



Luckily, Python makes it easy for us to expand the formula to n number of assets. This formula is better expressed in matrix notation since it's easier to apply this way in Python: 

$Portfolio Variance = Weightstransposed * (covariance matrix) * weights$

### 2.1 Defining Variables
Let's put those portfolio weights we calculated before into an array and also calculate the simple daily returns of our asset along with a covariance matrix of these daily returns. Keep in mind that **will multiply the daily returns covariance matrix by 252 to measure annual variance.**



In [17]:
weights = np.array([0.23,0.77])
#print(weights)
returns = prices.pct_change()
# pct_change:
# Computes the percentage change from the immediately previous row by default. This is useful in comparing the percentage of change in a time series of elements.
covariance = 252* returns.cov()
print(covariance)

Symbols        FB       CMG
Symbols                    
FB       0.262574  0.101437
CMG      0.101437  0.127265


After defining these, we simply need to take the dot product of the transposed weights with the dot product of the covariance matrix and weights to get the annual variance of our portfolio. Note: **The t-method in numpy just transposes the array.**

In [20]:
variance = np.dot(weights.T, np.dot(covariance, weights))

# Print the result
print(str(np.round(variance, 4) * 100) + "%")

12.53%


We determine that the annual variance of our portfolio is 10.33%. Let’s compare this to the variance of each individual asset.

### 2.2 Comparing Portfolio Variance to Individual Stock Variance
This shows the annual variance of a portfolio with FB and CMG weighted 23% and 77% respectively has shown a 10.33% annual variance over the last five years.

One extremely interesting byproduct of this result can be seen when calculating variance of the equities individually. One of the main pillars of modern portfolio theory is about how diversification reduces risk. We can see this clear as day by looking at each stock's variance over the same time period.